#### Mailchimp Automation

*Please note: This was written for a custom coded Mailchimp template. The script is custom built for this very specific template. It's probably not going to do anyone else any good, but it was a good learning exercise. Furthermore, the _only_ thing this script does is fill in text for a campaign. I still have to manually find images, add the headline and intro text for the email, etc. It saves me a lot of time every week and was worth every hair-pulling moment :)*

One last thing, you will need a webdriver in the same directory as your script to make this work. I used the Chrome driver, which can be found here: https://chromedriver.chromium.org/

Run the first block of code to login to Mailchimp. When the campaign is loaded, be sure to manually move the divider widget and scroll to the bottom of the template. This compensates for a Mailchimp frame sizing bug.

In [ ]:
import credentials # private login kept in separate py file
import csv
import os
import time
import selenium
from collections import namedtuple
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.supporat.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

### Event class contains basic info needed for newsletter events
Event = namedtuple('Event', 'event, date, desc, start_time, end_time, location, gcal_str')

### Finds the editor pane, switches to it, and enteres relevant text
def switch_to_editor():
    driver.switch_to.default_content()
    editor = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//iframe[@class="cke_wysiwyg_frame cke_reset"]'))
            )
    driver.switch_to.frame(editor)
    
def enter_event_name(index):
    editor_text = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//body'))
            )
    editor_text.send_keys(Keys.CONTROL + 'a')
    editor_text.send_keys(events[index].event)
    
def enter_event_desc(index):
    editor_text = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//body'))
            )
    editor_text.send_keys(Keys.CONTROL + 'a')
    editor_text.send_keys(events[index].desc)
    
def enter_event_date(index):
    editor_text = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//body'))
            )
    editor_text.send_keys(Keys.CONTROL + 'a')
    editor_text.send_keys(events[index].date)
    
### Finds and switches to the preview-template frame
### This is where we'll click the elements that need editing
def switch_to_preview_template():
    driver.switch_to.default_content()
    preview_template_frame = driver.find_element_by_xpath('//iframe[@id="preview-template"]')
    driver.switch_to.frame(preview_template_frame)
    
def find_name_blocks(block_name):
    return driver.find_elements_by_xpath(f"//*[contains(text(), '{block_name}')]")

def fill_event_names(events,events_length):    
    for i in range(len(events)):
        switch_to_preview_template()
        event_name_blocks[events_length-(i+1)].click()
        time.sleep(0.5)
        switch_to_editor()
        enter_event_name(-(i+1))
        #time.sleep(0.5)
        
def fill_event_descs(events,events_length):
    for i in range(len(events)):
        switch_to_preview_template()
        event_desc_blocks[events_length-(i+1)].click()
        time.sleep(0.5)
        switch_to_editor()
        enter_event_desc(-(i+1))
        #time.sleep(0.5)
        
def fill_event_dates(events,events_length):
    for i in range(len(events)):
        switch_to_preview_template()
        event_date_blocks[events_length-(i+1)].click()
        time.sleep(0.5)
        switch_to_editor()
        enter_event_date(-(i+1))
        #time.sleep(0.5)

        
### Change working directory, open CSV
### and create DictReader object of newsletter events
os.chdir('C:\\Users\\richa\\Documents\\Python Scripts')
file = ('newsletter_events.csv')
events = []
with open(file) as fh:
    rd = csv.DictReader(fh, delimiter=',') 
    for event in rd:
        events.append(Event(**event))
events_length = len(events)        
### Initiate driver and open Mailchimp
driver = webdriver.Chrome('C:\\Users\\richa\\Documents\\Python Scripts\\chromedriver')

driver.get("https://login.mailchimp.com")

u_name = driver.find_element_by_id("username")
u_name.send_keys(credentials.mc_name)
pw = driver.find_element_by_id("password")
pw.send_keys(credentials.mc_pw)

### Login will need to be manual on account of 2-factor

login_button = driver.find_element_by_xpath('//button[@type="submit"]')
login_button.click()

time.sleep(2)

verify_button = driver.find_element_by_xpath('//*[@role="button"]')
verify_button.click()

code_box = driver.find_element_by_id('sms-code')
code_box.send_keys(input('What is the verification code? '))
code_box.send_keys(Keys.ENTER)

campaign_id = input('What is the campaign ID? ')

driver.get(f'https://us19.admin.mailchimp.com/campaigns/wizard/neapolitan?id={campaign_id}')

#### On Campaign Load

Again, be sure to click and drag the divider between the preview template and the editor pane, then scroll to the bottom of the page. If you don't, Selenium will not be able to interact with the lowest content blocks, and the script will crash.

In [ ]:
switch_to_preview_template()
event_name_blocks = find_name_blocks('Intro Block')
event_desc_blocks = find_name_blocks('Lorem')
event_date_blocks = find_name_blocks('edate')
        
fill_event_names(events,events_length)
fill_event_dates(events,events_length)
fill_event_descs(events,events_length)

time.sleep(21) # Mailchimp autosaves every 20 seconds, so wait until we can be sure the info is saved

driver.quit()